In [0]:
dbutils.widgets.text("proc_date", "", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
%run ../../../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
create_temp_view_with_clean_columns(
    spark,
    catalog_name,
    'udp_wcm_bronze_sap_bw',
    'sap_bw_wcm_vccmdm35',
    proc_date,
    'temp_sap_vccmdm35'
)

In [0]:
spark.sql(f"""
CREATE TABLE {catalog_name}.udp_wcm_silver_sap_bw.sap_vccmdm35
(
  hash_id BIGINT,
  recordmode STRING,
  knumh STRING,
  knrmm DOUBLE,
  knrnm DOUBLE,
  knrme STRING,
  knrzm DOUBLE,
  knrez STRING,
  kappl STRING,
  kschl STRING,
  knrmat STRING,
  knrrr STRING,
  knrdd STRING,
  knr_ak STRING,
  kotabnr STRING,
  vakey STRING,
  proc_date DATE
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
) 
-- PARTITION BY DATE(proc_date);
""")

In [0]:
spark.sql(f"""
CREATE OR REPLACE TEMP VIEW temp_sap_vccmdm35 
AS
WITH temp AS (
SELECT  reqtsn
        ,datapakid
        ,record
        ,recordmode
        ,IF(knumh     = '',NULL,LTRIM('0', knumh)   ) knumh
        ,knrmm
        ,knrnm
        ,IF(knrme     = '',NULL,knrme   ) knrme
        ,knrzm
        ,IF(knrez     = '',NULL,knrez   ) knrez
        ,IF(kappl     = '',NULL,kappl   ) kappl
        ,IF(kschl     = '',NULL,kschl   ) kschl
        ,IF(knrmat     = '',NULL,LTRIM('0', knrmat)   ) knrmat
        ,IF(knrrr     = '',NULL,LTRIM('0', knrrr)   ) knrrr
        ,IF(knrdd     = '',NULL,LTRIM('0', knrdd)   ) knrdd
        ,IF(knr_ak     = '',NULL,LTRIM('0', knr_ak)   ) knr_ak
        ,IF(kotabnr     = '',NULL,LTRIM('0', kotabnr)   ) kotabnr
        ,IF(vakey     = '',NULL,vakey   ) vakey
        ,proc_date
         FROM (
                    SELECT * EXCEPT(rn) FROM (
                        SELECT *, ROW_NUMBER() OVER (PARTITION BY reqtsn, datapakid, record ORDER BY proc_date DESC) rn FROM  temp_sap_vccmdm35 
                    ) a WHERE rn = 1 
                ) a
),
main AS (
SELECT FARM_FINGERPRINT(CONCAT(
            IFNULL(CAST(knumh AS STRING), "")
        )) hash_id,
        * FROM temp
),
main_2 AS (
    SELECT * 
    , ROW_NUMBER() OVER (PARTITION BY hash_id ORDER BY reqtsn DESC,datapakid DESC,record  DESC) rn 
        FROM main
)
SELECT * EXCEPT(rn,reqtsn,datapakid,record) FROM main_2 WHERE rn = 1;
""")

In [0]:
spark.sql(f"""
MERGE INTO {catalog_name}.udp_wcm_silver_sap_bw.sap_vccmdm35 main_tbl
USING SAP_VCCMDM35 AS temp_tbl
ON main_tbl.hash_id = temp_tbl.hash_id 
WHEN NOT MATCHED THEN INSERT *
""")